Loading models based on validation loss and calculating test scores.

In [1]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
#https://www.amazon.com/Neural-Networks-Deep-Learning-Textbook/dp/3319944622/ref=cm_cr_arp_d_product_top?ie=UTF8
#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

/home/titli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [3]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [4]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [5]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [6]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
#print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of label tensor: (19997,)


In [7]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


# LSTM Model

In [8]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[10000:19000], ham_index[9997:19000]), axis=0)
keras_data_test_set = keras_data.iloc[new_index]
labels_test_set = keras_data_test_set.isspam
keras_data_test_set.head()

,Subject,isspam
19223,RE: Daily Summary of Risk Data,0
19224,Softwares CDS all software under 15$ and 99$!,1
19226,Hotel Room Bargains at up to 70% off! Save in...,1
19227,RE: Greeley Gas Company,0
19228,"Any Software just in under 15-99$, Xp-adobe etc",1


In [9]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_test_set.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_test_set = to_categorical(np.asarray(labels_test_set))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels_test_set = labels_test_set[indices]
x_test = data

Found 15425 unique tokens.
Shape of data tensor: (17088, 1000)
Shape of label tensor: (19997,)


In [10]:
embeddings_LSTM = pd.read_csv('embeddings_LSTM_model.csv')

In [11]:
embeddings_LSTM_sort = embeddings_LSTM.sort_values(['val_loss'])
embeddings_LSTM_sort.head(6)

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss,trainable_params
47,64,128,0,0,0,1,Nadam,0.931366,0.164316,0.928982,0.177332,251074
27,64,32,0,0,0,1,Nadam,0.928179,0.180042,0.924231,0.182697,38338
45,32,128,0,0,0,1,Nadam,0.926553,0.180566,0.922731,0.187457,242786
39,64,64,0,0,0,1,Nadam,0.933054,0.163492,0.926232,0.188538,92866
31,128,32,0,0,0,1,Nadam,0.925866,0.178017,0.926732,0.188866,42626
40,128,64,0,0,0,1,Adam,0.926491,0.177937,0.925231,0.189407,101250


In [12]:
def embedding_LSTM(fl1=16, fl2=16, fl3=16, dl=16, optimizer= 'RMSprop', kl = 5, layer =1): 
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Bidirectional(LSTM(units = fl1, return_sequences=True))(embedded_sequences)
    x = GlobalMaxPool1D()(x)
    x = Dense(units=dl, activation="relu")(x)
    x = Dropout(0.1)(x)
    preds = Dense(1, activation='tanh')(x)

    model = Model(sequence_input, preds)
    model.compile(loss= 'binary_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
    return model

## 1st model

In [13]:
kwargs = dict(fl1=128, fl2= 0, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_LSTM_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_LSTM_1.h5")
print("Saved model to disk")
model1_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 2496s 156ms/step - loss: 0.4724 - acc: 0.7567 - val_loss: 0.3248 - val_acc: 0.8245
Epoch 2/2
15998/15998 [==============================] - 3839s 240ms/step - loss: 0.3039 - acc: 0.8078 - val_loss: 0.2945 - val_acc: 0.8822
Saved model to disk
17088/17088 [==============================] - 633s 37ms/step


## 2nd model

In [14]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_LSTM_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_LSTM_2.h5")
print("Saved model to disk")
model2_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1400s 87ms/step - loss: 0.4546 - acc: 0.7660 - val_loss: 0.3861 - val_acc: 0.7202
Epoch 2/2
15998/15998 [==============================] - 1347s 84ms/step - loss: 0.3561 - acc: 0.7558 - val_loss: 0.3090 - val_acc: 0.7854
Saved model to disk
17088/17088 [==============================] - 300s 18ms/step


## 3rd model

In [15]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5, dl=128, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_LSTM_3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_LSTM_3.h5")
print("Saved model to disk")
model3_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1355s 85ms/step - loss: 0.5277 - acc: 0.6822 - val_loss: 0.4810 - val_acc: 0.6637
Epoch 2/2
15998/15998 [==============================] - 1321s 83ms/step - loss: 0.3620 - acc: 0.6833 - val_loss: 0.3529 - val_acc: 0.6909
Saved model to disk
17088/17088 [==============================] - 305s 18ms/step


## 4th model

In [16]:
kwargs = dict(fl1=64, fl2= 0, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_LSTM_4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_LSTM_4.h5")
print("Saved model to disk")
model4_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1729s 108ms/step - loss: 0.4451 - acc: 0.7417 - val_loss: 0.3650 - val_acc: 0.7684
Epoch 2/2
15998/15998 [==============================] - 1663s 104ms/step - loss: 0.3164 - acc: 0.8166 - val_loss: 0.3173 - val_acc: 0.8192
Saved model to disk
17088/17088 [==============================] - 376s 22ms/step


## 5th model

In [17]:
kwargs = dict(fl1=128, fl2= 0, fl3=0, kl=5, dl=32, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_LSTM_5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_LSTM_5.h5")
print("Saved model to disk")
model5_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 2690s 168ms/step - loss: 0.3989 - acc: 0.8246 - val_loss: 0.2714 - val_acc: 0.8982
Epoch 2/2
15998/15998 [==============================] - 2584s 162ms/step - loss: 0.2621 - acc: 0.9049 - val_loss: 0.2825 - val_acc: 0.9145
Saved model to disk
17088/17088 [==============================] - 571s 33ms/step


In [19]:
def F1_score(pred_test_y, actuals):

    predictions =[]
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0

    for i in range(len(pred_test_y)):
        if (pred_test_y[i]>=0.5):
            predictions.append(1)
        else:
            predictions.append(0)
    
    for i in range (len(pred_test_y)):
        if ((predictions[i]==1) & (actuals[i]==1)):
            true_pos = true_pos+1
        elif((predictions[i]==0) & (actuals[i]==0)):
            true_neg = true_neg+1
        elif((predictions[i]==1) & (actuals[i]==0)):
            false_pos = false_pos +1
        elif((predictions[i]==0) & (actuals[i]==1)):
            false_neg = false_neg+1
    prec=true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos+false_neg)
    accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
    F1=2*(prec*recall/(prec+recall))
    FPR = false_pos/(false_pos+true_neg)
    return (true_pos, false_pos, true_neg, false_neg, prec,FPR)

In [20]:
print((F1_score(model1_test_y, labels_test_set)))
print((F1_score(model2_test_y, labels_test_set)))
print((F1_score(model3_test_y, labels_test_set)))
print((F1_score(model4_test_y, labels_test_set)))
print((F1_score(model5_test_y, labels_test_set)))

(4864, 4312, 3751, 4161, 0.5300784655623365, 0.5347885402455662)
(5115, 4540, 3523, 3910, 0.5297773174520973, 0.5630658563809996)
(5271, 4779, 3284, 3754, 0.5244776119402985, 0.5927074289966514)
(4652, 4099, 3964, 4373, 0.5315963889841161, 0.5083715738558849)
(4806, 4268, 3795, 4219, 0.5296451399603262, 0.529331514324693)
